# 1. SQLite 데이터 베이스 연결 

##  (1) SQLite과 파이썬 
- SQLite 데이터 베이스는 별도의 서버프로세스 없이 SQL을 사용하여 DB액세스할 수 있도록 만든 간단한 **디스크 기반 데이터베이스**를 제공하는 C라이브러리
- SQLite는 프로토타입(시제품 나오기 전까지 제품)을 만들 때까지 사용. 정식 제품은 오라클이나 다른 DMBS를 이식.
- DB browser for SQLite(https://sqlitebrowser.org/dl)

In [1]:
import sqlite3
sqlite3.sqlite_version

'3.33.0'

## (2) 데이터 베이스 연결

In [5]:
# DB연결 객체 생성시 파일이 없으면 파일 생성, 파일이 없으면 그 파일에 연결
conn = sqlite3.connect('data/ch10_example.db')
conn

In [6]:
# 커서 객체 반환. 커서는 SQL문을 실행시키고 결과 데이터 조회.
cursor = conn.cursor()

In [11]:
cursor.execute("""
    CREATE TABLE MEMBERS_INFO (
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
""")

In [12]:
cursor.execute("INSERT INTO MEMBERS_INFO VALUES ('홍길동', 20, 'h@hong.com')")
cursor.execute("INSERT INTO MEMBERS_INFO VALUES ('최길동', 30, 'h@choi.com')")
cursor.execute("INSERT INTO MEMBERS_INFO VALUES ('김길동', 40, 'h@kim.com')")

In [ ]:
conn.commit()

In [13]:
cursor.execute('SELECT * FROM MEMBERS_INFO')

**select문의 결과를 받는 함수>** <br> 
- fetchall() - 결과를 모두 받을 때, 
- fetchone() - 결과를 한 행씩 받을 때, 
- fetchmay(n) - 결과를 n행 받을 때


In [10]:
print(cursor.fetchall())

[('홍길동', 20, 'h@hong.com'), ('최길동', 30, 'h@choi.com'), ('김길동', 40, 'h@kim.com')]


In [11]:
print(cursor.fetchall())

[]


In [14]:
cursor.execute("SELECT * FROM MEMBERS_INFO")
members = cursor.fetchall()
for member in members:
    print(member)

('홍길동', 20, 'h@hong.com')
('최길동', 30, 'h@choi.com')
('김길동', 40, 'h@kim.com')


In [15]:
cursor.execute('SELECT * FROM MEMBERS_INFO')
while True:
    member = cursor.fetchone()
    if member is None:
        break;
    print(member)

('홍길동', 20, 'h@hong.com')
('최길동', 30, 'h@choi.com')
('김길동', 40, 'h@kim.com')


In [16]:
cursor.execute('SELECT * FROM MEMBERS_INFO')
for member in cursor.fetchmany(2):
    print(member[0], member[1], member[2])

홍길동 20 h@hong.com
최길동 30 h@choi.com


In [17]:
cursor.fetchmany(2)

[('김길동', 40, 'h@kim.com')]

In [26]:
cursor.close() # 생략 가능
conn.close() # 반드시 실행

## (3) SQL구문에 파라미터 사용하기

### 1) qmark

In [18]:
cursor.execute("SELECT * FROM MEMBERS_INFO WHERE name IN ('홍길동','김철수')")
print(cursor.fetchall())

[('홍길동', 20, 'h@hong.com')]


In [21]:
cursor.execute("SELECT * FROM MEMBERS WHERE name IN (?,?)",('홍길동','김철수'))
print(cursor.fetchall())

[('홍길동', 20, 'h@hong.com')]


In [19]:
name1 = input('이름 검색1: ')
name2 = input('이름 검색2: ')
names = (name1, name2)

cursor.execute("SELECT * FROM MEMBERS_INFO WHERE NAME IN (?,?)", names)
print(cursor.fetchall())

이름 검색1: 김길동
이름 검색2: 박길동
[('김길동', 40, 'h@kim.com')]


In [20]:
name1 = input('이름 검색1: ')
name2 = input('이름 검색2: ')
names = [name1, name2]

cursor.execute("SELECT * FROM MEMBERS_INFO WHERE NAME IN (?,?)", names)
print(cursor.fetchall())

이름 검색1: 최길동
이름 검색2: 김길동
[('최길동', 30, 'h@choi.com'), ('김길동', 40, 'h@kim.com')]


### 2) named

In [24]:
name = input('이름 검색: ')
cursor.execute("SELECT * FROM MEMBERS_INFO WHERE NAME=:name",{'name':name})
members = cursor.fetchall()

if len(members) == 0:
    print('해당 이름의 데이터가 없습니다.')
else:
    print(cursor.fetchall())

이름 검색: 아무개
해당 이름의 데이터가 없습니다.


In [25]:
try:
    name = input('이름: ')
    age = int(input('나이: '))
    email = input('메일: ')
except:
    print('올바르지 않은 입력입니다. 초기값(20)으로 돌아갑니다.')
    age = 20
    email = input('메일: ')
cursor.execute("INSERT INTO MEMBERS_INFO VALUES(:name,:age,:email)",
              {'name':name, 'age':age, 'email':email})

이름: 박길동
나이: 30
메일: h@park.com


In [29]:
cursor.execute('SELECT * FROM MEMBERS_INFO')
print(cursor.fetchall())

[('홍길동', 20, 'h@hong.com'), ('최길동', 30, 'h@choi.com'), ('김길동', 40, 'h@kim.com'), ('박길동', 30, 'h@park.com')]


# 2. 오라클 데이터베이스 연결

In [51]:
# 데이터베이스 설정(DBMS가 있는 서버 주소, 포트번호, sid 등을 설정)
import cx_Oracle #prompt => pip install cx_Oracle 

oracle_dsn = cx_Oracle.makedsn(host='localhost', port=1521, sid='xe')
oracle_dsn

'(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))'

In [49]:
conn = cx_Oracle.connect("scott","tiger",dsn=oracle_dsn)
conn # 여기서 에러날 시 VC_redist.x64.exe 인스톨

<cx_Oracle.Connection to scott@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [45]:
# conn 얻어오는 방법1
oracle_dsn = cx_Oracle.makedsn(host='localhost', port=1521, sid='xe')
conn = cx_Oracle.connect("scott","tiger",dsn=oracle_dsn)

In [46]:
conn.close()

In [66]:
# conn 얻어오는 방법2
conn = cx_Oracle.connect("scott","tiger","localhost:1521/xe")
conn

<cx_Oracle.Connection to scott@localhost:1521/xe>

In [53]:
cursor = conn.cursor()

In [54]:
sql = "SELECT * FROM EMP"
cursor.execute(sql)
emp = cursor.fetchall()

In [55]:
for e in emp:
    print(e)

(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
(7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7900, 'JAMES', 'CL

In [56]:
sql = "select * from emp"
cursor.execute(sql)
for e in cursor:
    print(e)

(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
(7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7900, 'JAMES', 'CL

In [57]:
sql = "SELECT * FROM EMP WHERE DEPTNO =:deptno"
deptno = input('부서 번호: ')
cursor.execute(sql, {'deptno':deptno})

for emp in cursor:
    print(emp)

부서 번호: 10
(7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10)
(7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10)
(7934, 'MILLER', 'CLERK', 7782, datetime.datetime(1982, 1, 23, 0, 0), 1300.0, None, 10)


In [70]:
sql = "SELECT * FROM EMP WHERE DEPTNO=:deptno"
deptno = input('부서 번호: ')
cursor.execute(sql, {'deptno':deptno})

emp = cursor.fetchall()
if len(emp)==0 :
    print('입력하신 부서번호의 데이터가 없습니다.')
else:
    for e in emp:
        print(e)

부서 번호: 20
(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7902, 'FORD', 'ANALYST', 7566, datetime.datetime(1981, 12, 3, 0, 0), 3000.0, None, 20)


In [71]:
print(emp)

[(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20), (7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20), (7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20), (7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20), (7902, 'FORD', 'ANALYST', 7566, datetime.datetime(1981, 12, 3, 0, 0), 3000.0, None, 20)]


In [73]:
import pandas as pd

cursor.execute("SELECT * FROM EMP")
data = cursor.fetchall() # 튜플 리스트
data_df = pd.DataFrame(data) # 데이터 프레임
data_df

,0,1,2,3,4,5,6,7
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09,3000.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30


In [74]:
# 각 필드 특징 정보
# (필드명, type, display_size, 내부크기, 정확도, scale, nullable)
cursor.description

[('EMPNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 0),
 ('ENAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 10, 10, None, None, 1),
 ('JOB', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 9, 9, None, None, 1),
 ('MGR', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 1),
 ('HIREDATE', <cx_Oracle.DbType DB_TYPE_DATE>, 23, None, None, None, 1),
 ('SAL', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('COMM', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('DEPTNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 3, None, 2, 0, 1)]

In [75]:
[row[0] for row in cursor.description] 

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [76]:
data_df.columns = [row[0] for row in cursor.description] 

In [77]:
data_df.columns

Index(['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO'], dtype='object')

In [78]:
data_df.head()

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30


In [79]:
data_df.tail()

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30
10,7876,ADAMS,CLERK,7788.0,1983-01-12,1100.0,NaN,20
11,7900,JAMES,CLERK,7698.0,1981-12-03,950.0,NaN,30
12,7902,FORD,ANALYST,7566.0,1981-12-03,3000.0,NaN,20
13,7934,MILLER,CLERK,7782.0,1982-01-23,1300.0,NaN,10


In [80]:
cursor.close()
conn.close()

# 3. MariaDB 데이터베이스 연결

In [82]:
import pymysql # prompt => pip install pymysql

In [83]:
conn = pymysql.connect(host = 'localhost',
                      port = 3306,
                      db= 'park', user = 'root', passwd = 'mysql',
                      charset='utf8', autocommit=True)
conn

In [85]:
cursor = conn.cursor()
sql = "select * from personal"
cursor.execute(sql)
result = cursor.fetchall()

for r in result:
    print(r)

(1001, 'bill', 'president', None, datetime.date(1989, 1, 10), 7000, None, 10)
(1111, 'smith', 'manager', 1001, datetime.date(1990, 12, 17), 1000, None, 10)
(1112, 'ally', 'salesman', 1116, datetime.date(1991, 2, 20), 1600, 500, 30)
(1113, 'word', 'salesman', 1116, datetime.date(1992, 2, 24), 1450, 300, 30)
(1114, 'james', 'manager', 1001, datetime.date(1990, 4, 12), 3975, None, 20)
(1116, 'johnson', 'manager', 1001, datetime.date(1991, 5, 1), 3550, None, 30)
(1118, 'martin', 'analyst', 1111, datetime.date(1991, 9, 9), 3450, None, 10)
(1121, 'kim', 'clerk', 1114, datetime.date(1990, 12, 8), 4000, None, 20)
(1123, 'lee', 'salesman', 1116, datetime.date(1991, 9, 23), 1200, 0, 30)
(1226, 'park', 'analyst', 1111, datetime.date(1990, 1, 3), 2500, None, 10)


In [86]:
personal = pd.DataFrame(result)
personal.columns = [row[0] for row in cursor.description]
personal.head()

,pno,pname,job,manager,startdate,pay,bonus,dno
0,1001,bill,president,NaN,1989-01-10,7000,NaN,10
1,1111,smith,manager,1001.0,1990-12-17,1000,NaN,10
2,1112,ally,salesman,1116.0,1991-02-20,1600,500.0,30
3,1113,word,salesman,1116.0,1992-02-24,1450,300.0,30
4,1114,james,manager,1001.0,1990-04-12,3975,NaN,20


In [87]:
cursor.close()
conn.close()